# 01 - Sistema de Recomendación con LightFM (Filtrado Colaborativo)

Este notebook implementa un sistema de recomendación usando la librería `lightfm`, que permite usar filtrado colaborativo o híbrido con facilidad.

In [ ]:
# Instalación (si estás en Colab o no lo tienes instalado)
# !pip install lightfm

import pandas as pd
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k
import numpy as np

# Cargar datos
df = pd.read_csv('../data/ratings.csv')
df['rating'] = df['rating'].astype(float)

# Crear dataset en formato LightFM
dataset = Dataset()
dataset.fit(users=df['user_id'], items=df['item_id'])
interactions, _ = dataset.build_interactions([(x['user_id'], x['item_id'], x['rating']) for _, x in df.iterrows()])

# Crear modelo
model = LightFM(loss='warp')  # Alternativas: 'warp', 'bpr', 'logistic'
model.fit(interactions, epochs=10, num_threads=2)

# Evaluar el modelo
precision = precision_at_k(model, interactions, k=3).mean()
print(f"Precisión@3: {precision:.4f}")

In [ ]:
# Generar recomendaciones para cada usuario
def sample_recommendation(model, interactions, user_ids, dataset, num_items=3):
    n_users, n_items = interactions.shape
    item_labels = list(dataset.mapping()[2].keys())

    for user_id in user_ids:
        known_positives = [item_labels[i] for i in interactions.tocsr()[user_id].indices]
        scores = model.predict(user_id, np.arange(n_items))
        top_items = np.argsort(-scores)[:num_items]

        print(f"Usuario {user_id}")
        print("  Ya vio: ", known_positives)
        print("  Recomendado: ", [item_labels[i] for i in top_items])
        print("")

sample_recommendation(model, interactions, user_ids=[0, 1, 2], dataset=dataset)